In [ ]:
import requests
import json
import base64
import hmac
import hashlib
import datetime, time

with open('/mnt/c/python/API Keys/GEMINI_API_KEY.json') as f:
    api_key = json.load(f)
gemini_api_key = api_key['public_key']
gemini_api_secret = api_key['private_key'].encode()

In [ ]:
class gemini_kit(gemini_api_key, gemini_api_secret):
    
    def __init__(self, gemini_api_key, gemini_api_secret):
        self.gemini_api_key = gemini_api_key
        self.gemini_api_secret = gemini_api_secret.encode()
        self.base_url = "https://api.gemini.com"

    
    def place_order(self, symbol:str, amount:str, price:str, side:str, order_type:str="exchange limit", options:list=None, stop_price:str=None, account:str=None) -> dict:
        """
        symbol: 'btcusd', 'ethusd', etc...
        side: "buy", "sell"
        order_type: "exchange limit", "exchange stop limit"
        options (optional) list: "maker-or-cancel", "immediate-or-cancel", "fill-or-kill", "auction-only", "indication-of-interest"
        stop_price (optional)
        account (optional)
        """
        
        self.endpoint = "/v1/order/new"
        self.url = self.base_url + self.endpoint

        self.payload_nonce = str(int(time.mktime(datetime.datetime.now().timetuple()) * 1000))
        self.payload = {
            "request" : self.endpoint,
            "nonce": self.payload_nonce,
            "symbol": symbol,
            "amount": amount,
            "price": price,
            "side": side,
            "type": order_type
        }
        if options:
            self.payload["options"] = options
        elif stop_price:
            self.payload["stop_price"] = stop_price
        elif account:
            self.payload["account"] = account
        self.response = gemini_kit(gemini_api_key, gemini_api_secret).__send_payload(self)
        
        return self.response

    
        

    def cancel_order(self, order_id:str, all:bool or str=False, account:str=None) -> dict:
        """
        all (optional): True, False, 'session'
        account (optional)

        Cancels orders, if all=True it will cancel all open orders,
        if all='session' it will cancel all orders from session
        """
        if all is True:
            self.endpoint = '/v1/order/cancel/all'
        elif all.lower() is 'session':
            self.endpoint = '/v1/order/cancel/session'
        else:    
            self.endpoint = "/v1/order/cancel"
        
        self.url = self.base_url + self.endpoint
        self.payload_nonce = str(int(time.mktime(datetime.datetime.now().timetuple()) * 1000))

        self.payload = {
            "nonce" : self.payload_nonce,
            "order_id": order_id,
            "request" : self.endpoint
        }

        if account:
            self.payload['account'] = account

        self.response = gemini_kit(gemini_api_key, gemini_api_secret).__send_payload(self)
        
        return self.response


    def order_status(self, order_id:str=None, include_trades:bool=True, client_order_id:str=None, account:str=None) -> dict:
        """
        Gets status of orders
        """
        self.endpoint = "/v1/order/status"
        
        self.url = self.base_url + self.endpoint
        self.payload_nonce = str(int(time.mktime(datetime.datetime.now().timetuple()) * 1000))
        if order_id:
            self.order_id = order_id
        elif client_order_id:
            self.order_id = client_order_id
        else:
            raise ValueError('Must supply either order_id or client_order_id')

        self.payload = {
            "nonce" : self.payload_nonce,
            "order_id": self.order_id,
            "request" : self.endpoint
        }

        if account:
            self.payload['account'] = account
        
        self.response = gemini_kit(gemini_api_key, gemini_api_secret).__send_payload(self)
        
        return self.response


    def get_active_orders(self, account:str=None) -> list:
        """
        Gets status of all active orders
        """
        self.endpoint = "/v1/orders"
        
        self.url = self.base_url + self.endpoint
        self.payload_nonce = str(int(time.mktime(datetime.datetime.now().timetuple()) * 1000))

        self.payload = {
            "nonce" : self.payload_nonce,
            "request" : self.endpoint
        }

        if account:
            self.payload['account'] = account
        
        self.response = gemini_kit(gemini_api_key, gemini_api_secret).__send_payload(self)
        
        return self.response


    def get_past_trades(self, symbol:str, account:str=None) -> list:
        """
        symbol: 'btcusd', 'ethusd', etc...

        Gets past trades for given symbol
        """
        self.endpoint = "/v1/mytrades"
        
        self.url = self.base_url + self.endpoint
        self.payload_nonce = str(int(time.mktime(datetime.datetime.now().timetuple()) * 1000))

        self.payload = {
            "nonce" : self.payload_nonce,
            "request" : self.endpoint,
            "symbol": symbol
        }

        if account:
            self.payload['account'] = account
        
        self.response = gemini_kit(gemini_api_key, gemini_api_secret).__send_payload(self)
        
        return self.response


    def get_balances(self, account:str=None) -> list:
        """
        Gets current balances
        """
        self.endpoint = "/v1/balances"
        
        self.url = self.base_url + self.endpoint
        self.payload_nonce = str(int(time.mktime(datetime.datetime.now().timetuple()) * 1000))

        self.payload = {
            "nonce" : self.payload_nonce,
            "request" : self.endpoint
        }

        if account:
            self.payload['account'] = account
        
        self.response = gemini_kit(gemini_api_key, gemini_api_secret).__send_payload(self)
        
        return self.response


    def get_notional_balances(self, currency:str='USD', account:str=None) -> list:
        """
        Gets current balances
        """
        self.currency = currency.lower()
        self.endpoint = f"/v1/notionalbalances/{self.currency}"
        
        self.url = self.base_url + self.endpoint
        self.payload_nonce = str(int(time.mktime(datetime.datetime.now().timetuple()) * 1000))

        self.payload = {
            "nonce" : self.payload_nonce,
            "request" : self.endpoint
        }

        if account:
            self.payload['account'] = account
        
        self.response = gemini_kit(gemini_api_key, gemini_api_secret).__send_payload(self)
        
        return self.response


    def __send_payload(self) -> dict:
        self.encoded_payload = json.dumps(self.payload).encode()
        self.b64 = base64.b64encode(self.encoded_payload)
        self.signature = hmac.new(self.gemini_api_secret, self.b64, hashlib.sha384).hexdigest()

        self.request_headers = { 'Content-Type': "text/plain",
                    'Content-Length': "0",
                    'X-GEMINI-APIKEY': self.gemini_api_key,
                    'X-GEMINI-PAYLOAD': self.b64,
                    'X-GEMINI-SIGNATURE': self.signature,
                    'Cache-Control': "no-cache" }
        
        self.response = requests.post(self.url,
                        data=None,
                        headers=self.request_headers)
        self.response = self.response.json()
        
        return self.response
        

In [ ]:
gem = gemini_kit(gemini_api_key, gemini_api_secret)
order = gem.place_order('ethusd', .001, price, 'buy', 'exchange limit', ['maker-or-cancel'])